In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Mi pc no aguanta apriori :<, pruebo con mlxtend
from fim import *
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules, fpmax

In [33]:
df = pd.read_csv('./data/cleaned_online_retail.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [34]:
#A lo largo del tiempo vemos dos variaciones por cada mitad del año, por eso creo que separarlo en mitad tiene sentido sin complicar las cosas demas
# Creo que obtendremos mejores resultados porque estamos comparando dos periodos con cambios muy marcados por la etapa del a;o.
part1 = df[df['InvoiceDate'] < '2011-06-01']
part2 = df[df['InvoiceDate'] >= '2011-06-01']

In [35]:
def patternMining(df_oh, min_support, nd):
    freq = fpgrowth(df_oh, min_support=min_support, use_colnames=True)
    print("Itemsets frecuentes de la particion numero", nd)
    print(freq.head())
    rules = association_rules(freq, metric="lift", min_threshold=1)
    print("Reglas de asociacion de la particion numero", nd)
    print(rules[['support', 'confidence', 'lift']].head())
    maximal = fpmax(df_oh, min_support=min_support, use_colnames=True)
    print("Maximales de la particion numero", nd)
    print(maximal.head())
    return freq, rules, maximal

In [36]:
transactions1 = part1.groupby('InvoiceNo')['Description'].apply(list).tolist()
te = TransactionEncoder()
te_ary = te.fit(transactions1).transform(transactions1)
oneHot1 = pd.DataFrame(te_ary, columns=te.columns_)

transactions2 = part2.groupby('InvoiceNo')['Description'].apply(list).tolist()
te = TransactionEncoder()
te_ary = te.fit(transactions2).transform(transactions2)
oneHot2 = pd.DataFrame(te_ary, columns=te.columns_)


In [37]:
fre1, rules1, max1 = patternMining(oneHot1, 0.01, 1)
fre2, rules2, max2 = patternMining(oneHot2, 0.01, 2)

Itemsets frecuentes de la particion numero 1
    support                               itemsets
0  0.112946   (WHITE HANGING HEART T-LIGHT HOLDER)
1  0.022972  (KNITTED UNION FLAG HOT WATER BOTTLE)
2  0.020699       (CREAM CUPID HEARTS COAT HANGER)
3  0.019981                  (WHITE METAL LANTERN)
4  0.017229       (RED WOOLLY HOTTIE WHITE HEART.)
Reglas de asociacion de la particion numero 1
    support  confidence       lift
0  0.012443    0.110169   2.495384
1  0.012443    0.281843   2.495384
2  0.017349    0.496575  11.247633
3  0.017349    0.392954  11.247633
4  0.010529    0.093220   2.668273
Maximales de la particion numero 1
   support                            itemsets
0  0.01005   (UNION JACK FLAG PASSPORT COVER )
1  0.01005    (FELTCRAFT PRINCESS OLIVIA DOLL)
2  0.01005  (HANGING WOOD AND FELT BUTTERFLY )
3  0.01005     (12 PENCILS TALL TUBE WOODLAND)
4  0.01005        (RECYCLED ACAPULCO MAT PINK)
Itemsets frecuentes de la particion numero 2
    support                    

In [38]:
#support2 / support1

#df completo
merged_df = pd.merge(fre1, fre2, on='itemsets', how='outer', suffixes=('_1', '_2')).fillna(0)

#en los infinitos hay un JEP. Es la matrica que mas nos dice algo, especialmente considerando que estamos 
# evaluado etapas del a;o
merged_df['growth_rate'] = np.where(
    merged_df['support_1'] == 0, 
    np.inf, 
    merged_df['support_2'] / merged_df['support_1']
)

#emerging patterns si el GR > 1. Quitamos los inf porque son JEPs
eps = merged_df[(merged_df['growth_rate'] > 1) & (merged_df['growth_rate'] != np.inf)]
eps = eps.sort_values(by='growth_rate', ascending=False)

jeps = merged_df[merged_df['growth_rate'] == np.inf]
jeps = jeps.sort_values(by='support_2', ascending=False)

print("Emerging Patterns encontrados: " , len(eps))
print("Top Emerging Patterns:")
print(eps[['itemsets', 'support_1', 'support_2', 'growth_rate']].head())

print("Jumping Emerging Patterns encontrados: ", len(jeps))
print("Top Jumping Emerging Patterns:")
print(jeps[['itemsets', 'support_2', 'growth_rate']].head())

Emerging Patterns encontrados:  336
Top Emerging Patterns:
                                   itemsets  support_1  support_2  growth_rate
2130               (JUMBO BAG VINTAGE LEAF)   0.012324   0.059972     4.866456
1494               (HAND WARMER OWL DESIGN)   0.010888   0.041112     3.775957
1564      (PAPER CHAIN KIT 50'S CHRISTMAS )   0.019263   0.066672     3.461156
1606    (PAPER CHAIN KIT VINTAGE CHRISTMAS)   0.015315   0.047729     3.116577
1611  (SET OF 20 VINTAGE CHRISTMAS NAPKINS)   0.011606   0.035239     3.036336
Jumping Emerging Patterns encontrados:  1055
Top Jumping Emerging Patterns:
                                 itemsets  support_2  growth_rate
324   (GARDENERS KNEELING PAD KEEP CALM )   0.064439          inf
538          (SET OF 3 REGENCY CAKE TINS)   0.060303          inf
133          (HOT WATER BOTTLE KEEP CALM)   0.054843          inf
127            (JUMBO BAG VINTAGE DOILY )   0.054761          inf
343  (GARDENERS KNEELING PAD CUP OF TEA )   0.054430         

Se pueden identificar cosas interesantes y coherentes considerando la particion. Articulos navide;os como PAPER CHAIN KIT 50'S CHRISTMAS y demas han triplicado su presencia. Esto luego se vera mejor en graficas; esto nos podria indicar las fechas idoneas para empezar a promocionar estos productos. Ademas, tenemos un HAND WARMER OWL DESIGN por ejemplo, que nos indica el cambio de temporada; de igual manera, identificar en que fecha exactamente se produce su cambio para poder saber cuando promocionarlo; esto puede variar cada a;o debido a feonomenos meteorologicos. 

En cuanto a los JEps, estos nos indican mas que todo tendencias o modas virales; especialmente considerando que el dataset es del a;o 2011, cuando las redes sociales crecieron en popularidad. Tenemos nuevas lineas de producots como ------

Creo que podemos concluir que el dataset nos habla mucho acerca del ciclo estacional y tendencias muy marcadas. 